# RadioXenon Beta-Gamma Spectra Isotope Concentration Analysis
This research will focus on applying a few different neural networks to train a model that can accurately predict concentrations of the four main radioxenon isotopes/isomers from the beta-gamma spectra. The three models are a neural network (NN), a convolutional neural network (CNN), and then a hybrid network that is a mix of a convolutional neural network and direct input. This research will also test the behavior of the models under two different loss functions, Mean Squared Error (MSE) and Huber Loss.  Due to the complexity of the data utilized, this research will  focus on the models' performance on a Monte Carlo synthetically generated data set. Testing the models' behavior with a real data set is out of scope for this project.

It is meant to be run in Google Colab. I also did substantial work with the 'real data' from INL but I could not get the data to really be in the same form as the PNNL data. That work was super messy and so I just removed it from this code that I would post in relation to my work on the final project. The code I inherited had a ton of hard coded values that I was not sure where they came from or why, so I chose to just work with the PNNL data.

In [ ]:
#Connect the Google Drive
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
WorkingDirectory= r'/content/drive/MyDrive/Colab Notebooks/PHYS570AI/XeData/sims3'

In [ ]:
#Import Packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import math
import os, os.path
import scipy.optimize as opt
import scipy.integrate as integrate
from copy import deepcopy
from itertools import permutations
from numba import guvectorize, vectorize, cuda
from concurrent.futures import ThreadPoolExecutor
import time
import threading
from matplotlib import image
import mpmath as mp
import matplotlib as mpl
from datetime import datetime
import scipy.optimize as opt
from copy import deepcopy
from tqdm import tqdm

from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from __future__ import absolute_import, division, print_function, unicode_literals


from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ReduceLROnPlateau

from sklearn.decomposition import PCA
import seaborn as sns

import os
import re
import time
from pathlib import Path
from tqdm import tqdm

from tensorflow.keras.losses import Huber


In [ ]:
#functions

# -------------------------
# TestAccumulator
# -------------------------
class TestAccumulator:
    def __init__(self, workingDir):
        self.results = []
        self.workingDir = str(workingDir)

    def execute(self, use_tqdm=True, save_cache=None):
        """
        Walk self.workingDir, build XeFile objects and cache if requested.
        - use_tqdm: show progress bar
        - save_cache: path (str) to .npz file to save histograms+concentrations
        """
        start = time.time()
        files = []
        for root, _, fnames in os.walk(self.workingDir):
            for fname in fnames:
                files.append(os.path.join(root, fname))

        loader = (tqdm(files, desc="Processing XeFiles") if use_tqdm else files)

        self.results = []
        for p in loader:
            try:
                self.results.append(XeFile(p))
            except Exception as e:
                # don't stop the whole run for one bad file; log it instead
                print(f"Failed to load {p}: {e}")

        elapsed = time.time() - start
        print(f"Processed {len(self.results)} files in {elapsed:.1f} s")

        if save_cache:
            # Save arrays for quick reload: histograms and concentrations
            hist_list = []
            conc_list = []
            for r in self.results:
                # ensure histogram is numpy array
                hist_list.append(np.asarray(r['histogram']))
                conc_list.append(np.asarray(r.concentration, dtype=np.float64))
            # Stack histograms if shapes agree; otherwise save as object array
            try:
                hist_stack = np.stack(hist_list)
            except Exception:
                hist_stack = np.array(hist_list, dtype=object)
            conc_stack = np.vstack(conc_list)
            np.savez_compressed(save_cache, histograms=hist_stack, concentrations=conc_stack)
            print(f"Saved cache to {save_cache}")

        return self.results


# -------------------------
# XeFile
# -------------------------
class XeFile(dict):
    """
    Minimal API preserved:
      - instance behaves like dict with keys for blocks (renamed)
      - attributes: filename, concentration (4-tuple list), getEnergies(), histogram, histogramUncertainty, etc.
    """
    # patterns used for concentration extraction
    FLOAT_RE = re.compile(r'(?P<num>\d+(?:\.\d+)?)')
    # useful labels to detect compound filenames; kept to preserve legacy mapping logic
    COMPOUND_TAGS = [
        "Xe131m_Xe133m_Xe133_Xe135",
        "Xe131m_Xe133m_Xe133",
        "Xe131m_Xe133_Xe135",
        "Xe133andXe133m",
        "Xe131m_Xe135",
        "Xe131m_Xe133",
        "Xe133_Xe135",
        "Xe133m_Xe133_Xe135",
        "Xe131m",
        "Xe133",
        "Xe135",
    ]

    def __init__(self, arg):
        dict.__init__(self)
        if isinstance(arg, str):
            self.filename = arg
            self.loadFromPHD(arg)
        elif isinstance(arg, dict):
            # preserve old behavior if dict passed
            dict.update(self, arg)
            # best-effort set concentration if present
            self.concentration = list(dict.get(self, "concentration", [0, 0, 0, 0]))
        else:
            raise Exception("XeFile requires either a string file path or a dictionary as its argument.")

    # small fast numeric conversion; tries to avoid exceptions where possible
    @staticmethod
    def _string_to_num_token(tok):
        # quick heuristic: has decimal point or exponent -> float
        if any(c in tok for c in ".eE"):
            try:
                return float(tok)
            except ValueError:
                return tok
        else:
            # integer-like token
            try:
                return int(tok)
            except ValueError:
                try:
                    return float(tok)
                except ValueError:
                    return tok

    def _parse_line_tokens(self, line):
        # split by whitespace and convert tokens
        parts = line.strip().split()
        return [self._string_to_num_token(tok) for tok in parts]

    def loadFromPHD(self, filePath):
        """
        Stream the file once, build raw block text, then parse only what is required.
        Preserves full parsing behavior.
        """
        # helper to get filename base
        filename = os.path.basename(filePath)

        # -----------------------
        # 1) extract concentration(s)
        # -----------------------
        # default
        conc1 = conc2 = conc3 = conc4 = 0.0

        # find floating numbers in filename
        nums = [float(m.group('num')) for m in self.FLOAT_RE.finditer(filename)]


        # Intent: if a specific tag is present, map positions appropriately
        # Assume numbers found correspond to the concentrations in order.
        if any(tag in filePath for tag in ["Xe131m_Xe133m_Xe133_Xe135"]):
            if len(nums) >= 4:
                conc1, conc2, conc3, conc4 = nums[:4]
        elif any(tag in filePath for tag in ["Xe131m_Xe133m_Xe133", "Xe131m_Xe133_Xe135",
                                             "Xe131m_Xe135", "Xe131m_Xe133", "Xe133_Xe135",
                                             "Xe133m_Xe133_Xe135"]):
            if len(nums) >= 4:
                conc1, conc2, conc3, conc4 = nums[:4]
        elif "Xe133andXe133m" in filePath:
            # special two-number filename form
            if len(nums) >= 2:
                conc2, conc3 = nums[:2]
        elif "Xe131m" in filePath and not ("Xe131m_" in filePath):
            # single isotope filename, like "12nXYZ.phd" -> interpreted earlier as single conc
            if nums:
                conc1 = nums[0]
        elif "Xe133" in filePath and not ("Xe133_" in filePath):
            if nums:
                conc2 = nums[0]
        elif "Xe135" in filePath and not ("Xe135_" in filePath):
            if nums:
                conc4 = nums[0]
        else:
            # fallback: if 4 numbers present assume them
            if len(nums) >= 4:
                conc1, conc2, conc3, conc4 = nums[:4]
            elif len(nums) == 2:  # maybe a two-isotope file
                conc2, conc3 = nums[:2]

        self.concentration = [conc1, conc2, conc3, conc4]

        # -----------------------
        # 2) stream file into blocks (single pass)
        # -----------------------
        blocks_raw = {}         # map block_name -> list[str] (body lines)
        current_key = None
        with open(filePath, 'r') as fh:
            for raw_line in fh:
                line = raw_line.rstrip('\n')
                if not line:
                    continue
                if line.startswith('#'):
                    # new block header: everything after '#' stripped
                    current_key = line[1:].strip()
                    # initialize block list (may get overwritten later)
                    blocks_raw[current_key] = []
                else:
                    if current_key is None:
                        # lines before first '#' are ignored by original; we skip
                        continue
                    blocks_raw[current_key].append(line)

        # store filepath
        self.update({'filePath': filePath})

        # -----------------------
        # 3) convert raw blocks into lists of tokenized lines (numbers/strings)
        # -----------------------
        # Reuse parsed tokens to avoid repeated splits later
        parsed_blocks = {}
        for k, lines in blocks_raw.items():
            parsed = [self._parse_line_tokens(l) for l in lines if l.strip() != '']
            parsed_blocks[k] = parsed

        # attach parsed blocks as original code would have (but keep parsed tokens)
        # Keep tokenized lists.
        self.update(parsed_blocks)

        # -----------------------
        # 4) Build structured blocks per original 'blocks' map
        # -----------------------
        blocks_map = [
            {'oldName': 'g_Energy', 'newName': 'gammaEnergyCalibration', 'keys': ['energy', ['channel', ['value','uncertainty']]]},
            {'oldName': 'b_Energy', 'newName': 'betaEnergyCalibration', 'keys': ['energy', 'type', ['channel',['value','uncertainty']]]},
            {'oldName': 'g_Resolution', 'newName': 'gammaResolution', 'keys': ['energy', ['fwhm', ['value','uncertainty']]]},
            {'oldName': 'b_Resolution', 'newName': 'betaResolution', 'keys': ['energy', ['fwhm', ['value','uncertainty']]]},
            {'oldName': 'g_Efficiency', 'newName': 'gammaEfficiency', 'keys': ['energy', ['efficiency', ['value','uncertainty']]]},
            {'oldName': 'b-gEfficiency', 'newName': 'betaEfficiency', 'keys': ['nuclide', 'roi', ['efficiency', ['value','uncertainty']]]},
            {'oldName': 'Ratios', 'newName': 'ratios', 'keys': ['identifier', 'roiHigher', 'roiLower', ['ratio', ['value','uncertainty']]]},
            {'oldName': 'ROI_Limits', 'newName': 'roiEnergyRange', 'keys': ['roi', 'blower', 'bupper', 'glower', 'gupper']},
        ]

        for b in blocks_map:
            old = b['oldName']
            new = b['newName']
            raw = self.pop(old, [])  # may be [] if missing
            out_list = []
            for line in raw:
                lineDict = {}
                for i, key in enumerate(b['keys']):
                    if isinstance(key, list):
                        # nested map: key[0] -> dict of subkeys from key[1]
                        subname = key[0]
                        subkeys = key[1]
                        # map subkeys with consecutive items from line
                        subdict = {subkeys[j]: line[i + j] for j in range(len(subkeys))}
                        lineDict[subname] = subdict
                    else:
                        lineDict[key] = line[i] if i < len(line) else None
                out_list.append(lineDict)
            self[new] = out_list

        # -----------------------
        # 5) clean ROI block into dict-of-dicts
        # -----------------------
        roi_raw = self.pop('roiEnergyRange', [])
        try:
            self['roiEnergyRange'] = {
                roi['roi']: {'beta': {'lower': roi['blower'], 'upper': roi['bupper']},
                             'gamma': {'lower': roi['glower'], 'upper': roi['gupper']}}
                for roi in roi_raw
            }
        except Exception:
            # fallback empty
            self['roiEnergyRange'] = {}

        # -----------------------
        # 6) Build spectra (beta/gamma) as pandas Series
        #    This requires getEnergies() to have been run first because it uses calibration blocks
        # -----------------------
        # Ensure energy calibrations exist; if not, leave spectra empty
        if ('gammaEnergyCalibration' in self) and ('betaEnergyCalibration' in self):
            # compute energies & uncertainties
            self.getEnergies()
            spectraType = {'beta': 'b', 'gamma': 'g'}
            for key in spectraType:
                # original code popped e.g. 'b_Spectrum' and took [1:]
                blockname = spectraType[key] + '_Spectrum'
                spectrum = self.pop(blockname, [])[1:]
                # original code removed first item of each line (index), so drop index token if present
                # and flatten remaining tokens into one big array
                if spectrum:
                    cleaned = []
                    for line in spectrum:
                        # drop first token (index) if it exists
                        if line:
                            cleaned.append(line[1:])
                    if cleaned:
                        arr = np.concatenate(cleaned).ravel()
                        self[key + 'Spectrum'] = pd.Series(arr, index=self[key + 'Energy'].values)
                    else:
                        self[key + 'Spectrum'] = pd.Series(dtype=float)
                else:
                    self[key + 'Spectrum'] = pd.Series(dtype=float)
        else:
            # keep placeholders if no calibration present
            self['betaSpectrum'] = pd.Series(dtype=float)
            self['gammaSpectrum'] = pd.Series(dtype=float)

        # -----------------------
        # 7) Build histogram DataFrame & uncertainties
        # -----------------------
        hist_raw = self.pop('Histogram', [])
        if hist_raw:
            # original used histogram[1:-1] - keep same indexing behavior
            if len(hist_raw) >= 2:
                histogram_rows = hist_raw[1:-1]
            else:
                histogram_rows = hist_raw

            # convert to numpy 2D array of floats (guard against ragged)
            try:
                N = np.array(histogram_rows, dtype=float)
            except Exception:
                # ragged -> convert each row individually into an array then stack with padding
                rows = [np.asarray(r, dtype=float) for r in histogram_rows]
                maxc = max(len(r) for r in rows)
                padded = np.vstack([np.pad(r, (0, maxc - len(r)), 'constant', constant_values=0.0) for r in rows])
                N = padded

            # dx, dy and uncertainties come from previously built keys (they are numbers in the original parser)
            # original code used: dx = self['betaEnergyDelta']; dy = self['gammaEnergyDelta']
            dx = float(self.get('betaEnergyDelta', 1.0))
            dy = float(self.get('gammaEnergyDelta', 1.0))
            sigma_dx = float(self.get('betaEnergyDeltaUncertainty', 0.0))
            sigma_dy = float(self.get('gammaEnergyDeltaUncertainty', 0.0))

            # preserve old DataFrame semantics for indexing and columns (gamma energies are rows, beta energies columns)
            # But store histogram as numpy array for speed; keep DataFrame if user expects it in self
            try:
                beta_energy_vals = self['betaEnergy'].values if isinstance(self['betaEnergy'], pd.Series) else np.arange(N.shape[1])
            except Exception:
                beta_energy_vals = np.arange(N.shape[1])
            try:
                gamma_energy_vals = self['gammaEnergy'].values if isinstance(self['gammaEnergy'], pd.Series) else np.arange(N.shape[0])
            except Exception:
                gamma_energy_vals = np.arange(N.shape[0])

            # store both numpy and pandas-like object as in original code
            self['histogram'] = pd.DataFrame(N / (dx * dy),
                                             index=gamma_energy_vals,
                                             columns=beta_energy_vals)
            # also keep a numpy mirror for faster numeric ops
            self.histogram = np.array(N / (dx * dy))

            # compute uncertainty preserving original formula
            sqrtN = np.sqrt(N)
            with np.errstate(divide='ignore', invalid='ignore'):
                term1 = (sqrtN / (dx * dy)) ** 2
                term2 = (-N * sigma_dx / (dx ** 2 * dy)) ** 2
                term3 = (-N * sigma_dy / (dx * dy ** 2)) ** 2
                term4 = 0.1 / (dx * dy)
                hist_unc = np.sqrt(term1 + term2 + term3 + term4)

            self['histogramUncertainty'] = pd.DataFrame(hist_unc,
                                                        index=gamma_energy_vals,
                                                        columns=beta_energy_vals)
        else:
            self['histogram'] = pd.DataFrame()
            self.histogram = np.array([])
            self['histogramUncertainty'] = pd.DataFrame()

        return self

    # -----------------------
    # getEnergies method
    # -----------------------
    def getEnergies(self, plotEnergyChannelFit=False):
        """
        Computes self['betaEnergy'], self['gammaEnergy'], their uncertainties, min/max etc.
        Returns (betaEnergySeries, gammaEnergySeries)
        """
        for energyType in ['betaEnergy', 'gammaEnergy']:
            calib_key = energyType + 'Calibration'
            if calib_key not in self or not self[calib_key]:
                # no calibration; create default linear mapping
                chan = pd.Series(range(1, 257), index=range(1, 257))
                self[energyType] = chan
                self[energyType + 'Uncertainty'] = pd.Series(np.zeros_like(chan), index=chan.index)
                self[energyType + 'Delta'] = 1.0
                self[energyType + 'DeltaUncertainty'] = 0.0
                self[energyType + 'Min'] = float(self[energyType].iloc[0])
                self[energyType + 'Max'] = float(self[energyType].iloc[-1])
                continue

            energies = np.array([data['energy'] for data in self[calib_key]], dtype=float)
            channels = np.array([data['channel']['value'] for data in self[calib_key]], dtype=float)
            uncertainties = np.array([data['channel'].get('uncertainty', 1e-6) or 1e-6 for data in self[calib_key]], dtype=float)

            # linear fit E = dE*chan + E0 with weights = 1/uncertainties
            # use numpy.polyfit with cov
            (dE, E0), covariance = np.polyfit(channels, energies, 1, w=1.0 / uncertainties, cov=True)

            self[energyType + 'Delta'] = float(dE)
            self[energyType + 'DeltaUncertainty'] = float(np.sqrt(covariance[0, 0]))
            chan = pd.Series(range(1, 257), index=range(1, 257))
            self[energyType] = chan * self[energyType + 'Delta'] + E0
            # uncertainty formula preserved
            cov00 = covariance[0, 0]
            cov11 = covariance[1, 1]
            cov01 = covariance[0, 1]
            self[energyType + 'Uncertainty'] = np.sqrt((chan * cov00) ** 2 + cov11 ** 2 + 2 * chan * abs(cov01))
            self[energyType + 'Min'] = float(self[energyType].iloc[0])
            self[energyType + 'Max'] = float(self[energyType].iloc[-1])

            if plotEnergyChannelFit:
                plt.figure(figsize=(4, 4))
                x = np.arange(1, 101)
                y = self[energyType][:100]
                yminus = y - self[energyType + 'Uncertainty'][:100]
                yplus = y + self[energyType + 'Uncertainty'][:100]
                plt.plot(x, y)
                plt.plot(x, yminus, linestyle='dashed')
                plt.plot(x, yplus, linestyle='dashed')
                plt.errorbar(channels, energies, yerr=uncertainties, fmt='o')
                plt.xlabel('Channel')
                plt.ylabel(energyType)
                plt.show()

        return (self['betaEnergy'], self['gammaEnergy'])


# -------------------------
# Helper plotting functions
# -------------------------
def plotProjection(data, spectrumObject=None, figureScale=1, resolution=500):
    # (kept almost identical to original but minor robustness updates)
    if isinstance(spectrumObject, list):
        x = np.linspace(data.columns[0], data.columns[-1], resolution)
        y = np.linspace(data.index[0], data.index[-1], resolution)
        xmesh, ymesh = np.meshgrid(x, y)
        spectrumObject = pd.DataFrame(spectrum(spectrumObject, xmesh, ymesh), columns=x, index=y)

    if isinstance(spectrumObject, pd.DataFrame):
        dx = spectrumObject.columns[1] - spectrumObject.columns[0]
        dy = spectrumObject.index[1] - spectrumObject.index[0]
        xSpectrum = spectrumObject.sum(0) * dy
        ySpectrum = spectrumObject.sum(1) * dx
    else:
        xSpectrum = None
        ySpectrum = None

    dataDx = data.columns[1] - data.columns[0]
    dataDy = data.index[1] - data.index[0]

    axesRatios = np.array([data.columns[-1] - data.columns[0], data.index[-1] - data.index[0]])
    axesRatios = 3 * axesRatios / max(axesRatios)
    figureSize = figureScale * 2 * (1 + axesRatios) + np.array([2, 0])

    fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2, constrained_layout=True, figsize=figureSize,
                                                 gridspec_kw={'height_ratios': [axesRatios[1], 1],
                                                              'width_ratios': [1, axesRatios[0]]})

    plot(data, spectrumObject, axes=ax2, xLabel=None, yLabel=None)
    plot(data.sum(0) * dataDy, xSpectrum, axes=ax4, xLabel=r'$E_\beta$ (keV)')
    dataGamma = data.sum(1) * dataDx
    ax1.errorbar(dataGamma.values, dataGamma.index, xerr=np.sqrt(dataGamma.values), fmt='.')
    if ySpectrum is not None:
        y = ySpectrum.index
        ax1.plot(ySpectrum.values, y)
    ax1.invert_xaxis()
    ax1.margins(0.05, 0)
    ax1.set_ylabel(r'$E_\gamma$ (keV)')
    ax3.axis('off')
    ax3.text(0, 0, 'Counts / keV', rotation=0)
    ax4.set_ylabel('')
    return [fig, [[ax1, ax2], [ax3, ax4]]]



import os
import re
import time
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_histogram(file_path):
    """
    Unified loader for both MC (.phd) and real INL (.pbg) histograms.
    BOTH are 256×256 for your dataset.
    """

    with open(file_path, "r", errors="ignore") as f:
        lines = [l.strip() for l in f.readlines()]

    # Locate histogram block
    start = None
    stop = None
    for i, line in enumerate(lines):
        if "#Histogram" in line:
            start = i
        if ("STOP" in line) or ("#Certificate" in line):
            stop = i
            break

    if start is None or stop is None:
        raise ValueError("Histogram block not found")

    hist_block = lines[start+1 : stop]

    # First line contains metadata for real files (256 256 700 1000)
    # Determine whether this file has metadata based on how many numbers are in line 0.
    meta_vals = hist_block[0].split()

    if len(meta_vals) == 4:
        # REAL format (.pbg)
        data_lines = hist_block[1:]
        expected_shape = (256, 256)
        file_type = "real"
    else:
        # MC format (.phd)
        data_lines = hist_block
        expected_shape = (256, 256)
        file_type = "MC"

    # Convert to matrix
    mat = np.array([[float(v) for v in row.split()] for row in data_lines], dtype=np.float32)

    # Validate shape
    if mat.shape != expected_shape:
        raise ValueError(f"Unexpected histogram shape: {mat.shape}, expected: {expected_shape}")

    # Normalize
    total = mat.sum()
    if total > 0:
        mat /= total

    return mat, {"type": file_type}


import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cosine
from scipy.stats import entropy

# -----------------------------------------------------
# Preprocessing: log transform + normalize to probability
# -----------------------------------------------------
def preprocess_hist(mat):
    mat = np.asarray(mat, dtype=np.float32)

    # ensure non-negative
    mat = np.clip(mat, 0, None)

    # log transform to control big peaks
    mat = np.log1p(mat)

    # Normalize to probability
    s = mat.sum()
    if s > 0:
        mat = mat / s

    return mat


def preprocess_for_model(hist):
    """Input: (256,256) or (256,256,1) histogram
       Output: processed histogram with same shape."""

    # Remove channel dimension if present
    if hist.ndim == 3:
        hist = hist[:, :, 0]

    # clip negatives
    hist = np.clip(hist, 0, None)

    # log transform
    hist = np.log1p(hist)

    # normalize to total counts (probability map)
    s = hist.sum()
    if s > 0:
        hist = hist / s

    return hist





In [ ]:
# Change default plotting parameters
mpl.rcParams['image.cmap'] = 'bone_r'
mpl.rcParams['image.origin'] = 'lower'
plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams['font.size'] = 14


In [ ]:
#Pre-process MC, PNNL Data, only need to run this once and then the data will be good to just recall from wherever it is stored --saves time
bob = TestAccumulator(WorkingDirectory)
bob.execute() #(WorkingDirectory)

conclist = [result.concentration for result in bob.results]
histlist = [result.histogram for result in bob.results]

histData = np.asarray(histlist, dtype=np.uint16)
concData = np.asarray(conclist, dtype=np.uint32)


save_path = os.path.join(WorkingDirectory, "processed_results.npz")
np.savez_compressed(save_path, histData=histData, concData=concData)
print("Saved:", save_path)




In [ ]:
#To load in pre-processed MC Data
data = np.load(os.path.join(WorkingDirectory, "processed_results.npz"))
histData = data["histData"]
concData = data["concData"]

In [ ]:
#Shared Set up and Preprocessing
#Prepare train/val/test splits and reshape

# histData: (N, 256, 256) from your TestAccumulator
# concData: (N, 4)         isotope counts

# cast to float32 for TF
histData_f = histData.astype("float32")
#histData_f = np.array([preprocess_for_model(h) for h in histData], dtype=np.float32)

concData_f = concData.astype("float32")

# single global train/test split
X_train_val, X_test, y_train_val, y_test = train_test_split(
    histData_f, concData_f, test_size=0.2, random_state=42
)

# for CNN / Combined (need channel dimension)
X_train_val_img = X_train_val[..., np.newaxis]  # (N, 256, 256, 1)
X_test_img      = X_test[..., np.newaxis]

# for NN (flat input)
X_train_val_flat = X_train_val.reshape(len(X_train_val), -1)  # (N, 256*256)
X_test_flat      = X_test.reshape(len(X_test), -1)

# train/validation split
X_train_img, X_val_img, y_train, y_val = train_test_split(
    X_train_val_img, y_train_val, test_size=0.2, random_state=42
)
X_train_flat, X_val_flat, _, _ = train_test_split(
    X_train_val_flat, y_train_val, test_size=0.2, random_state=42
)

# shared callbacks
redlr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)


def plot_history(history, title_prefix=""):
    """Plot loss and MAE for a Keras History object."""
    h = history.history
    epochs = range(1, len(h["loss"]) + 1)

    plt.figure(figsize=(12,4))

    # Loss
    plt.subplot(1,2,1)
    plt.plot(epochs, h["loss"], label="train")
    if "val_loss" in h:
        plt.plot(epochs, h["val_loss"], label="val")
    plt.xlabel("Epoch")
    plt.ylabel("MSE loss")
    plt.title(f"{title_prefix} Loss")
    plt.legend()
    plt.grid(True)

    # MAE
    if "mae" in h:
        plt.subplot(1,2,2)
        plt.plot(epochs, h["mae"], label="train")
        if "val_mae" in h:
            plt.plot(epochs, h["val_mae"], label="val")
        plt.xlabel("Epoch")
        plt.ylabel("MAE")
        plt.title(f"{title_prefix} MAE")
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()



#Model Training


Here is where you would change the loss to MSE or Huber.... also change where the model and outputs are being saved if you want to run both, so that the results for both are saved separately

In [ ]:
# ------------------------------------------------
# 2. NN model (flatten → dense layers)
# ------------------------------------------------
nn_input = layers.Input(shape=(256*256,))
x = layers.Dense(128, activation="relu")(nn_input)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
nn_output = layers.Dense(4, activation="linear")(x)

modelNN = models.Model(nn_input, nn_output)
modelNN.compile(optimizer="adam", loss=Huber(delta=1.0), metrics=["mae", "mse"])
modelNN.summary()

historyNN = modelNN.fit(
    X_train_flat, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_flat, y_val),
    callbacks=[redlr, early],
    verbose=1
)

plot_history(historyNN, title_prefix="NN")

# predictions on test set if you want
test_predictionsNN = modelNN.predict(X_test_flat)


In [ ]:
# === Saving ===

# Save predictions

np.save(
    os.path.join(WorkingDirectory, "test_predictionsNN_new_loss.npy"),
    test_predictionsNN,
    allow_pickle=True
)


# Save model

save_dir = os.path.join(WorkingDirectory, "saved_models_new_loss")
os.makedirs(save_dir, exist_ok=True)

modelNN.save(os.path.join(save_dir, "modelNN_new_loss.keras"))


# Save training history

with open(os.path.join(WorkingDirectory, "training_historyNN_new_loss.json"), "w") as f:
    json.dump(historyNN.history, f)

print("Model, predictions, and history saved successfully.")


In [ ]:
#Load in NN results if want to do analysis without retraining



# Paths
model_path = os.path.join(WorkingDirectory, "saved_models_new_loss", "modelNN_new_loss.keras")
pred_path  = os.path.join(WorkingDirectory, "test_predictionsNN_new_loss.npy")
hist_path  = os.path.join(WorkingDirectory, "training_historyNN_new_loss.json")

#load model
loaded_modelNN = tf.keras.models.load_model(model_path)
print("Loaded model:", loaded_modelNN)

#predictions
loaded_predictionsNN = np.load(pred_path, allow_pickle=True)
print("Loaded predictions shape:", loaded_predictionsNN.shape)

#training history
with open(hist_path, "r") as f:
    loaded_historyNN = json.load(f)

print("Loaded history keys:", loaded_historyNN.keys())


In [ ]:
#CNN Model

cnn_input = layers.Input(shape=(256, 256, 1))

c = layers.Conv2D(32, (3,3), activation="relu", padding="same")(cnn_input)
c = layers.MaxPooling2D((2,2))(c)
c = layers.Conv2D(64, (3,3), activation="relu", padding="same")(c)
c = layers.MaxPooling2D((2,2))(c)
c = layers.Conv2D(128, (3,3), activation="relu", padding="same")(c)
c = layers.MaxPooling2D((2,2))(c)

c = layers.Flatten()(c)
c = layers.Dense(128, activation="relu")(c)
c = layers.Dense(64,  activation="relu")(c)
cnn_output = layers.Dense(4, activation="linear")(c)

modelCNN = models.Model(cnn_input, cnn_output)
modelCNN.compile(optimizer="adam", loss=Huber(delta=1.0), metrics=["mae", "mse"])
modelCNN.summary()

historyCNN = modelCNN.fit(
    X_train_img, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_img, y_val),
    callbacks=[redlr, early],
    verbose=1
)

plot_history(historyCNN, title_prefix="CNN")

test_predictionsCNN = modelCNN.predict(X_test_img)


In [ ]:
# === Saving ===

#save predictions
np.save(
    os.path.join(WorkingDirectory, "test_predictionsCNN_new_loss.npy"),
    test_predictionsCNN,   # <-- no need to wrap in list
    allow_pickle=True
)

#save model
save_dir = os.path.join(WorkingDirectory, "saved_models_new_loss")
os.makedirs(save_dir, exist_ok=True)

modelCNN.save(os.path.join(save_dir, "modelCNN_new_loss.keras"))  # <-- only one model object

#save training history
with open(os.path.join(WorkingDirectory, "training_historyCNN_new_loss.json"), "w") as f:
    json.dump(historyCNN.history, f)   # <-- history is not a list anymore

print("Model, predictions, and history saved successfully.")


In [ ]:
#Load in CNN

model_path = os.path.join(WorkingDirectory, "saved_models_new_loss", "modelCNN_new_loss.keras")
pred_path  = os.path.join(WorkingDirectory, "test_predictionsCNN_new_loss.npy")
hist_path  = os.path.join(WorkingDirectory, "training_historyCNN_new_loss.json")

#model
loaded_modelCNN = tf.keras.models.load_model(model_path)
print("Loaded model:", loaded_modelCNN)

#predictions
loaded_predictionsCNN = np.load(pred_path, allow_pickle=True)
print("Loaded predictions shape:", loaded_predictionsCNN.shape)

#training history
with open(hist_path, "r") as f:
    loaded_historyCNN = json.load(f)

print("Loaded history keys:", loaded_historyCNN.keys())


In [ ]:
#Combined model (CNN branch + direct image branch)

# CNN branch
comb_cnn_input = layers.Input(shape=(256, 256, 1), name="cnn_input")
z = layers.Conv2D(32, (3,3), activation="relu", padding="same")(comb_cnn_input)
z = layers.MaxPooling2D((2,2))(z)
z = layers.Conv2D(64, (3,3), activation="relu", padding="same")(z)
z = layers.MaxPooling2D((2,2))(z)
z = layers.Conv2D(128, (3,3), activation="relu", padding="same")(z)
z = layers.MaxPooling2D((2,2))(z)
z = layers.Flatten()(z)

# "existing data" branch (right now also the histogram image)
comb_existing_input = layers.Input(shape=(256, 256, 1), name="existing_input")
y_flat = layers.Flatten()(comb_existing_input)

# Concatenate learned features + raw image info
combined = layers.concatenate([z, y_flat])

# Dense head
h = layers.Dense(256, activation="relu")(combined)
h = layers.Dense(128, activation="relu")(h)
h = layers.Dense(64,  activation="relu")(h)
comb_output = layers.Dense(4, activation="linear")(h)

modelCombined = models.Model(
    inputs=[comb_cnn_input, comb_existing_input],
    outputs=comb_output
)
modelCombined.compile(optimizer="adam", loss=Huber(delta=1.0), metrics=["mae", "mse"])
modelCombined.summary()

historyCombined = modelCombined.fit(
    [X_train_img, X_train_img],  # both branches get the same data for now
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=([X_val_img, X_val_img], y_val),
    callbacks=[redlr, early],
    verbose=1
)

plot_history(historyCombined, title_prefix="Combined")

test_predictionsCombined = modelCombined.predict([X_test_img, X_test_img])


In [ ]:
#Save Combined

#save predictions
np.save(
    os.path.join(WorkingDirectory, "test_predictionsCombined_new_loss.npy"),
    test_predictionsCombined,   # <-- no need to wrap in list
    allow_pickle=True
)

#save model
save_dir = os.path.join(WorkingDirectory, "saved_models_new_loss")
os.makedirs(save_dir, exist_ok=True)

modelCombined.save(os.path.join(save_dir, "modelCombined_new_loss.keras"))  # <-- only one model object

#save training history
with open(os.path.join(WorkingDirectory, "training_historyCombined_new_loss.json"), "w") as f:
    json.dump(historyCombined.history, f)   # <-- history is not a list anymore

print("Model, predictions, and history saved successfully.")


In [ ]:
#Open Combined

model_path = os.path.join(WorkingDirectory, "saved_models_new_loss", "modelCombined_new_loss.keras")
pred_path  = os.path.join(WorkingDirectory, "test_predictionsCombined_new_loss.npy")
hist_path  = os.path.join(WorkingDirectory, "training_historyCombined_new_loss.json")

#load model
loaded_modelCombined = tf.keras.models.load_model(model_path)
print("Loaded model:", loaded_modelCombined)

#load predictions
loaded_predictionsCombined = np.load(pred_path, allow_pickle=True)
print("Loaded predictions shape:", loaded_predictionsCombined.shape)

#load training history
with open(hist_path, "r") as f:
    loaded_historyCombined = json.load(f)

print("Loaded history keys:", loaded_historyCombined.keys())


#Evaluation

In [ ]:
#if using the loaded in models
modelNN      = loaded_modelNN
modelCNN     = loaded_modelCNN
modelCombined= loaded_modelCombined

In [ ]:
#Concentration Plotting
isotope_idx = {
    "Xe131m": 0,
    "Xe133": 1,
    "Xe133m": 2,
    "Xe135": 3
}

pred_CNN   = modelCNN.predict(X_test_img)
pred_NN    = modelNN.predict(X_test_flat)
pred_comb  = modelCombined.predict([X_test_img, X_test_img])

for name, idx in isotope_idx.items():

    plt.figure(figsize=(5,5))
    plt.scatter(y_test[:, idx], pred_CNN[:, idx],  alpha=0.3, label="CNN")
    plt.scatter(y_test[:, idx], pred_NN[:, idx],   alpha=0.3, label="NN")
    plt.scatter(y_test[:, idx], pred_comb[:, idx], alpha=0.3, label="Combined")

    plt.xlabel("True Concentration")
    plt.ylabel("Predicted Concentration")
    plt.title(f"Prediction vs True — {name}")
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
#confusion rates

def confusion_rate(y_true, y_pred, threshold=0.005):
    """
    Computes % of wrong-isotope predictions.

    Wrong isotope = true = 0 AND predicted > threshold_value.

    threshold: expressed as fraction of max true concentration.
    """
    # Compute a dynamic threshold per isotope to ensure it scales with different magnitudes of concentrations
    # Count zero truth (when isotope is truly absent) cases and false positives
    max_per_isotope = np.max(y_true, axis=0)
    thresh_vals = threshold * max_per_isotope   # shape (4,)

    N = len(y_true)
    wrong_counts = np.zeros(4)
    total_zeros  = np.zeros(4)

    for iso in range(4):
        true_zero = (y_true[:, iso] == 0) #boolean mask
        total_zeros[iso] = np.sum(true_zero) #number of negatives

        pred_wrong = y_pred[:, iso] > thresh_vals[iso] #predicts above threshold
        wrong_counts[iso] = np.sum(pred_wrong & true_zero) #flase positives

    rate = wrong_counts / total_zeros
    return rate, wrong_counts, total_zeros, thresh_vals


# ---- Compute rates for all 3 models ----
rates_CNN, wrong_CNN, N0_CNN, threshCNN = confusion_rate(y_test, loaded_predictionsCNN)
rates_NN, wrong_NN, N0_NN, threshNN = confusion_rate(y_test, loaded_predictionsNN)
rates_COMB, wrong_COMB, N0_COMB, threshCOMB = confusion_rate(y_test, loaded_predictionsCombined)

iso_names = ["Xe131m", "Xe133m", "Xe133", "Xe135"]

print("\n=== WRONG ISOTOPE CONFUSION RATES ===")
for i, name in enumerate(iso_names):
    print(f"\n{name}:")
    print(f"  CNN:      {rates_CNN[i]*100:.2f}%   ({wrong_CNN[i]}/{N0_CNN[i]})")
    print(f"  NN:       {rates_NN[i]*100:.2f}%   ({wrong_NN[i]}/{N0_NN[i]})")
    print(f"  Combined: {rates_COMB[i]*100:.2f}%   ({wrong_COMB[i]}/{N0_COMB[i]})")
